In [22]:
require 'countries'

false

# 80. コーパスの整形

文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
空文字列となったトークンは削除
以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [23]:
doc = []
pat = "[#{["\\", "\\.", "\\,", "\\!", "\\?", "\\;", "\\:", "\\(", "\\)", "\\[", "\\]", "\\'", "\""].join()}]"

File.foreach('../data/enwiki_small').with_index do |line, index|
#File.foreach('../data/enwiki-20150112-400-r100-10576.txt').with_index do |line, index|
  p index if index % 50000 == 0
  sentence = line.chomp.split.map {|token|
    token.
      sub(/^#{pat}/, '').
      sub(/#{pat}$/, '').
      gsub(/#{pat}/, '') # ズル
    }.
    reject{|token| token.empty?}
  
  doc << sentence.join("\s") if sentence.size > 5
end

1

0


1

# 81. 複合語からなる国名への対処

英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は“United States”，イギリスは“United Kingdom”と表現されるが，“United”や“States”，“Kingdom”という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．
インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，“United States”は“United_States”，“Isle of Man”は“Isle_of_Man”になるはずである．

In [24]:
original = ISO3166::Country.all.map{|country| country.name.sub(/\s\(.*?\)\s/, '')}.select{|name| name.match(/\s/)}
under_scored = original.map{|name| name.gsub(/\s/, '_')}

doc.each_with_index do |sentence, index|
  p index if index % 50000 == 0
  original.size.times do |i|
    sentence.gsub!(original[i-1], under_scored[i-1])
  end
end

1

0


1

# 82. 文脈の抽出

81で作成したコーパス中に出現するすべての単語tに関して，単語tと文脈語cのペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．
- ある単語tの前後d単語を文脈語cとして抽出する（ただし，文脈語に単語tそのものは含まない）
- 単語tを選ぶ度に，文脈幅dは{1,2,3,4,5}の範囲でランダムに決める．

In [25]:
def window
  (Random.rand * 4.9999).to_i + 1
end

output = open("../data/context.tsv", "w")

doc.each_with_index do |words, index|
 p index if index % 50000 == 0
  words = words.split(/\s|\n/)
  
  words.size.times.each do |i|
    w = window

    start_from = [0, i - w].max
    end_to = [words.size-1, i + w].min

    contexts = words[start_from...i] + words[(i+1)..end_to]
    contexts.each do |context|
      output.puts "#{words[i]}\t#{context}"
    end
  end
  break
end

output.close

1

1

# 83. 単語／文脈の頻度の計測

82の出力を利用し，以下の出現分布，および定数を求めよ．

f(t,c)f(t,c): 単語ttと文脈語ccの共起回数
f(t,∗)f(t,∗): 単語ttの出現回数
f(∗,c)f(∗,c): 文脈語ccの出現回数
NN: 単語と文脈語のペアの総出現回数

In [26]:
count_t = Hash.new{|hash, key| hash[key] = 0}
count_c = Hash.new{|hash, key| hash[key] = 0}
count_t_c = Hash.new{|hash, key| hash[key] = Hash.new{|hash2, key2| hash2[key2] = 0}}
n = 0

File.foreach("../data/context.tsv").with_index do |line, index|
  p index if index % 500000 == 0
  t, c = line.split
  count_t[t] += 1
  count_c[c] += 1
  count_t_c[t][c] += 1
  n += 1
end

1

0
